In [1]:
import anndata as ad
import pandas as pd
import scanpy as sc
import scvi
import numpy as np
from lightning.pytorch import seed_everything

seed_everything(12345)
scvi.settings.seed = 12345

/Users/ljiang/anaconda3/envs/scanpy_scvi_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 12345
Global seed set to 12345


In [2]:
out_dir = "analysis/adata_obj/"
adata = sc.read_h5ad(out_dir+"/adata.h5ad")

In [3]:
scvi.model.SCVI.setup_anndata(
    adata, 
    layer="counts", 
    batch_key='SampleName',
)

arches_params = dict(
    n_latent = 10, 
    dropout_rate=0.1, 
    n_layers=2, 
    gene_likelihood="zinb"
)

model = scvi.model.SCVI(adata, **arches_params)

In [4]:
model.train(
    #accelerator="cpu", 
    #devices=4,
    max_epochs=100, 
    check_val_every_n_epoch=1,
    early_stopping=False, 
    enable_progress_bar=True
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Epoch 100/100: 100%|█| 100/100 [18:25:53<00:00, 737.44s/it, v_num=1, train_loss_

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 100/100: 100%|█| 100/100 [18:25:53<00:00, 663.54s/it, v_num=1, train_loss_


In [ ]:
model

In [ ]:
model.history.keys()

In [5]:
pd.DataFrame(model.history['validation_loss']).to_csv(out_dir+"validation_loss.csv")
pd.DataFrame(model.history['reconstruction_loss_train']).to_csv(out_dir+"reconstruction_loss_train.csv")
pd.DataFrame(model.history['reconstruction_loss_validation']).to_csv(out_dir+"reconstruction_loss_validation.csv")
pd.DataFrame(model.history['elbo_validation']).to_csv(out_dir+"elbo_validation.csv")
pd.DataFrame(model.history['elbo_train']).to_csv(out_dir+"elbo_train.csv")

In [6]:
adata.obsm['X_scVI'] = model.get_latent_representation(adata).astype(np.float32)

In [7]:
model.save(out_dir, overwrite=True)
adata.write(out_dir+"adata.h5ad", compression="gzip")